# How to find answers using jupyter?

Include this header script in your notebooks to find more answers.

How to use inheritence in jupyter notebooks?

How to make a class out of jupter notebooks?

How to use jupyter notebooks as services?

How to sanity check the environment?


In [1]:
// initialize

// What do we end up with?
var importNotebook = (notebook) => {};
var getCells = ((notebook, types = ['code']) => Promise.resolve([]));
var logIm = (cells, filename) => {};
var runAllInNEL = (cells) => Promise.resolve([]);
var assignResults = (results, filename) => [];

// provide a function for importing any notebook as a module and executing it
var importNotebook = (notebook) => {
    try {
        return getCells(notebook, 'code')
            .then(cells => runAllInNEL(cells))
            .then(results => assignResults(results, notebook))
            .catch(e => {
            console.log(e);
            return Promise.resolve([]);
        });
    } catch(e){console.log(e); return Promise.resolve([])}
};

// What does import return?
if (typeof imported == 'undefined') { var imported = {}; }
imported['How to find answers.ipynb'] = Promise.resolve({
    'How to find answers.ipynb[0]': importNotebook,
    'How to find answers.ipynb[1]': getCells
});

// How to test if a notebook has already been imported?
var oldImport = importNotebook;
var testImport = (notebook) => {
    var filename = path.basename(notebook);
    if(typeof notebook === 'undefined') {
        return Promise.resolve([]);
    }
    if (typeof imported[filename] !== 'undefined') {
        console.log('Already imported');
        console.log(Object.keys(imported));
        return imported[filename];
    }
    return oldImport(notebook);
};

// test to make sure it works
$$.async();
testImport('How to find answers.ipynb')
    .then((results) => (importNotebook = testImport))
    .then(() => $$.sendResult(importNotebook))
    .catch((e) => $$.sendError(e));

// $$.done(); for our dumb parser



Already imported
[ 'How to find answers.ipynb' ]


[Function: testImport]

In [2]:
// initialize

// How to parse cells of {types} from a notebook?
var streamJson = (file, match = ((c) => true)) => {
    var cells = [];
    var file = fs.createReadStream(file)
        .pipe(JSONStream.parse('cells.*'));
    file.on('data', (cell) => {
        if(match(cell)) {
            cells[cells.length] = cell.source.join('').trim();
        }
    });
    // create a promise out of this stream
    return new Promise((resolve, reject) => {
        file.on('error', e => reject(e, file));
        file.on('close', () => resolve(cells, file))
    });
};

// How to use getCells?
var getCells = (notebook, types = ['code']) => {
    return streamJson(notebook, (cell) => 
                      types.indexOf(cell.cell_type) > -1);
};

// $$.done() for our dumb parser


In [3]:
// initialize

// How to run all promises sequentially?
var runAllPromises = (promises) => {
    return promises.reduce((promise, func) => {
        return promise.then(result => {
            return (new Promise(func))
                .then(Array.prototype.concat.bind(result));
        });
    },Promise.resolve([]));
};

// How to create VM promises out of the cell content?
var runAllInNEL = (cells) => {
    var sess = new nel.Session();
    return runAllPromises(
        cells.map(
            (cell) => 
            (resolve) => 
            sess.execute(cell, {
                onSuccess: (o) => resolve(o),
                onError: (o) => resolve(o),
                beforeRun: () => {}})
    ));
};



In [4]:
// initialize
// cache results when notebooks are processed
var assignResults = (results, notebook) => {
    var filename = path.basename(notebook);
    var results = results.reduce((obj, r, i) => {
        if (typeof r === 'function') {
            obj[filename+'['+i+']'] = r
        }
        return obj;
    }, {});
    Object.assign(global, results);
    imported[filename] = Promise.resolve(results);
    return results;
};

// display a heading when new notebooks are imported
var logIm = (cells, filename) => {
    return 'Running ' + filename  
                + ' with ' + cells.length + ' cells';
    console.log();
    return ;
};
// $$.done() for our dumb parser


How to import a {notebook}?

How to test the import feature?


In [2]:

// initialize
try {
    // How to test to make sure the cell parser works
    var execSync = require('child_process').execSync;
    var deps = ['npm install', 'JSONStream', 'nel', 'path', 'fs', 'vm'];
    var assign = v => Object.assign(global[v] = require(v));
    execSync(deps.slice(0, 3).join(' '), {stdio: 'inherit'});
    deps.slice(1).forEach(assign);
    
    // How to test to make sure the VM works?
    $$.async();
    importNotebook('How to test import.ipynb')
        .then(() => $$.done('done importing'));

}catch(e){$$.sendError(e)}

// $$.done() for our dumb parser


'done importing'

In [6]:
// $$.done()


These are some other useful notebooks you may want to import.

How to train a chatbot using jupyter services?

What is code completion for jupter notebooks?



In [3]:
//$$.async();
//importNotebook('How to interpret questions.ipynb')
//    .then(() => $$.done('done importing importing'));

$$.async();

setTimeout(() => $$.sendResult('hit2'), 2000);
// $$.done()
